In [3]:
import numpy as np 
import pandas as pd 
from sklearn.cluster import KMeans
import matplotlib as plt


In [5]:
# Data simulation 
np.random.seed(4)

# Data frame to represent students
fields = ['Artificial Intelligence', 'Computer Science','Software Engineering','Electrical Engineering','Business Analytics']
batches = [19,20,21,22]
room_capacities = [25, 30, 30, 35]

# Students data 
students = pd.DataFrame ({'Student_ID': range(2450), 
                          'Field':np.random.choice(fields,2450),
                          'Batch':np.random.choice(batches,2450)})

# Rooms data 
rooms = pd.DataFrame({'Room_ID':range(1,31),
                      'Capacity':np.random.choice(room_capacities,30) })

# Faculty data 
faculty = pd.DataFrame({'Faculty_ID':range(1,51),
                        'Expertise':np.random.choice (fields,50)})

      Student_ID                    Field  Batch
0              0     Software Engineering     19
1              1         Computer Science     21
2              2  Artificial Intelligence     21
3              3  Artificial Intelligence     20
4              4     Software Engineering     21
...          ...                      ...    ...
2445        2445         Computer Science     21
2446        2446         Computer Science     19
2447        2447     Software Engineering     20
2448        2448         Computer Science     22
2449        2449         Computer Science     21

[2450 rows x 3 columns]
    Room_ID  Capacity
0         1        30
1         2        30
2         3        35
3         4        30
4         5        30
5         6        30
6         7        30
7         8        25
8         9        30
9        10        35
10       11        25
11       12        35
12       13        25
13       14        25
14       15        30
15       16        30
16       17  